In [1]:
import pandas as pd 
import numpy as np
import os

pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)

In [2]:
# linear_table = pd.read_csv('matching_table/mono_match_linear_count.csv')


linear_table = pd.read_csv('matching_table/mono_match_linear_v2.csv')

nonlinear_table = pd.read_csv('matching_table/mono_match_nonlinear_v2.csv' )

linear_table['type'] = 'linear'
nonlinear_table['type'] = 'nonlinear'

all_table = pd.concat([linear_table, nonlinear_table])
all_table.index = range(len(all_table))


In [3]:
all_mono = all_table['csv'].values
all_pdb = all_table['pdb'].values
all_type = all_table['type'].values
all_glycan_name = all_table['glycan name'].values
# all_count = all_table['size'].values

In [4]:
all_unique_csv = np.unique(all_mono)

In [5]:
def sorting(lst):
    lst2 = sorted(lst, key=len, reverse=True)
    return lst2
glycan_list = ['gal', 'galnac', 'galn', 'gala', 'glc', 'glcnac', 'glcn', 'glca', 'fuc', 
               'man', 'mannac', 'mann', 'mana', 'kdn', 'neu5ac', 'xyl', 'idoa', 'neu5gc']

glycan_list_extend = ['gal', 'galnac', 'galn', 'gala', 'galanac', 'glc', 'glcnac', 'glcn', 'glca', 'fuc', 
                      'man', 'mannac', 'mann', 'mana', 'kdn', 'neu5ac', 'xyl', 'idoa', 'neu5gc', 
                      'kdo', 'kdn', 'quip',  'quinac', 'rhap', 'ara', 'gula', 'rib', 'neu5ac', 'tal', 'rha', 'rhanac',
                      'fucnac', 'rhanac', 'ido']

glycan_list_extend = sorting(glycan_list_extend)
print(glycan_list_extend)

['galanac', 'galnac', 'glcnac', 'mannac', 'neu5ac', 'neu5gc', 'quinac', 'neu5ac', 'rhanac', 'fucnac', 'rhanac', 'galn', 'gala', 'glcn', 'glca', 'mann', 'mana', 'idoa', 'quip', 'rhap', 'gula', 'gal', 'glc', 'fuc', 'man', 'kdn', 'xyl', 'kdo', 'kdn', 'ara', 'rib', 'tal', 'rha', 'ido']


In [6]:
all_unique_csv

array(['A-D-FUCP', 'A-D-FUCPNAC', 'A-D-GALF', 'A-D-GALNAC', 'A-D-GALP',
       'A-D-GALPA', 'A-D-GALPA2SO3', 'A-D-GALPAN', 'A-D-GALPASO3',
       'A-D-GALPNAC', 'A-D-GLC', 'A-D-GLCP', 'A-D-GLCP1OME', 'A-D-GLCPA',
       'A-D-GLCPN', 'A-D-GLCPN1PO4', 'A-D-GLCPNAC', 'A-D-KDOP',
       'A-D-MANP', 'A-D-MANP1SET', 'A-D-MANP2AC', 'A-D-MANP2AC1SET',
       'A-D-MANP3AC', 'A-D-MANP4AC', 'A-D-MANP6AC', 'A-D-QUIP4N',
       'A-D-RHAP', 'A-D-RHAP4NAC', 'A-L-ARAF', 'A-L-ARAP', 'A-L-FUCP',
       'A-L-FUCP2SO3', 'A-L-FUCP2SO33SO34SO3', 'A-L-FUCP2SO34SO3',
       'A-L-FUCP3SO34SO3', 'A-L-FUCPNAC', 'A-L-GALP', 'A-L-GULPA',
       'A-L-RHAP', 'B-D-FUCPNAC', 'B-D-GALF', 'B-D-GALF2AC',
       'B-D-GALFOAC', 'B-D-GALP', 'B-D-GALP1OME', 'B-D-GALPA',
       'B-D-GALPNAC', 'B-D-GLC1OME', 'B-D-GLCP', 'B-D-GLCP1OME',
       'B-D-GLCP2AC', 'B-D-GLCPA', 'B-D-GLCPAN', 'B-D-GLCPN',
       'B-D-GLCPN4PO4', 'B-D-GLCPNAC', 'B-D-GLCPNAC6SO3', 'B-D-KDOP',
       'B-D-MANP', 'B-D-MANP2AC', 'B-D-MANPA1NAC3NAC', 'B-D-MA

In [7]:
def split_single_mono_bound(mono):
    
    mono = mono.lower()
    
    ab = 'missing_a_b'
    ld = 'missing_L_D'
    abbrev = ''
    
    _ct = mono.count('-')
    temp_mono_list = mono.split('-')
    if _ct == 2:
        
        
        ab = temp_mono_list[0]
        ld = temp_mono_list[1]
        abbrev = temp_mono_list[2]
        
    elif _ct == 1:
#         the following are wired only one appeared
#         D-GLCPA
#         D-GLCPN
#         DELTA-GLCPAN
        temp_bound_incomp = temp_mono_list[0]
        
        if temp_bound_incomp in ['delta', 'd', 'l']:
            ld = temp_bound_incomp 
            if ld == 'delta':
                ld = 'd'
        elif temp_bound_incomp in ['a', 'b']:
            ab = temp_bound_incomp 
        
        abbrev = temp_mono_list[1]
        
    elif _ct == 0:
        abbrev = mono
    
    elif _ct == 3:
#         the following are wired, three bound appeared, all treated as A-D-HEPP
#         D-A-D-HEPP
#         L-A-D-HEPP
#         L-A-D-HEPP 
        ab = temp_mono_list[1]
        ld = temp_mono_list[2]
        abbrev = temp_mono_list[3]
    elif _ct == 4:
#         the following are wired, four bound appeared,
#         L-gro-a-D-manHepp
#         a-L-6-deoxy-Talp
        if 'man' in mono:
            ab = 'a'
            ld = 'd'
            abbrev = 'manhepp'
        if 'tal' in mono:
            ab = 'a'
            ld = 'l'
            abbrev = 'talp'
    if abbrev == 'monosaccharid':
        abbrev = 'missing_monosaccharide'
    
    return ab, ld, abbrev

In [8]:
def reformulate_monosaccharide(abbrev, g_list):
    
    if 'p' in abbrev:
        carbon_number = 'P'
        current_split_list = abbrev.split('p')
    
    elif 'f' in abbrev:
        carbon_number = 'F'
        current_split_list = abbrev.split('f')
    
    else:
        carbon_number = 'missing_carbon_number'
        current_split_list = [abbrev]
#     print(current_split_list)
    new_abbrev = ''.join(current_split_list)
    
    indicator = 0
#     print(new_abbrev)
    for m in g_list:
        if (m in new_abbrev) and (indicator == 0):
            
            new_mono = m
            indicator = 1
            prev_m = m
        elif (m in new_abbrev) and (indicator == 1):
            
            if len(m) > len(prev_m):
                new_mono = m
            else:
                new_mono = prev_m
        
    if indicator == 0:
        new_mono = 'missing_monosaccharide'
    
    return abbrev, new_mono, carbon_number

In [9]:
''.join(['gal', 'nac'])

'galnac'

In [10]:
reformulate_monosaccharide('galpnac', glycan_list_extend)

('galpnac', 'galnac', 'P')

In [11]:
ab_list = []
ld_list = []
abbrev_list = []
pf_list = []

standard_abbrev_list = []

for i in range(len(all_mono)):
    current_mono = all_mono[i]
    ab, ld, abbrev = split_single_mono_bound(current_mono)
#     print(ab, ld, abbrev)
    ab_list.append(ab)
    ld_list.append(ld)
    abbrev_list.append(abbrev)
    if ('p' not in abbrev) and ('f' not in abbrev):
#         print(i, abbrev, all_glycan_name[i], all_type[i], all_pdb[i])
        if abbrev == 'glc1ome':
            abbrev = 'glcp'
        
        if abbrev == 'galnac':
            abbrev = 'galpnac'
        
        if abbrev == 'glc':
            abbrev = 'glcp'
            
        if abbrev == 'kdn':
            abbrev = 'kdnp'
            
        if abbrev == 'tyr':
            abbrev = 'tyrp'
    
    if 'p' in abbrev:
        current_split_list = abbrev.split('p')
    
    elif 'f' in abbrev:
        current_split_list = abbrev.split('f')
    
    else:
        current_split_list = [abbrev]
    if len(current_split_list) > 2:
        pass
#         print(current_split_list, abbrev)
    
#     if any(ext in abbrev for ext in glycan_list_extend):
#         pass
#     else:
#         print(abbrev)
    abbrev, new_mono, carbon_number = reformulate_monosaccharide(abbrev, glycan_list_extend)
    print(abbrev, new_mono, carbon_number)
    
    standard_abbrev_list.append(new_mono)
    pf_list.append(carbon_number)

glcpnac glcnac P
kdop kdo P
kdop kdo P
kdop kdo P
fucp fuc P
galpnac galnac P
galpnac galnac P
glcpa glca P
glcp glc P
xylp xyl P
galp gal P
galpnac galnac P
glcp glc P
glcpnac glcnac P
glcp glc P
glcp glc P
rhap4nac rha P
rhap4nac rha P
rhap4nac rha P
glcp1ome glc P
galp gal P
missing_monosaccharide missing_monosaccharide missing_carbon_number
glcp glc P
manpnac mannac P
galp gal P
glcpa glca P
glcpnac glcnac P
glcpnac glcnac P
glcpnac glcnac P
galpa gala P
galpa gala P
glcp glc P
glcp glc P
glcpnac glcnac P
glcp glc P
manpnac mannac P
glcp glc P
glcpa glca P
missing_monosaccharide missing_monosaccharide missing_carbon_number
missing_monosaccharide missing_monosaccharide missing_carbon_number
missing_monosaccharide missing_monosaccharide missing_carbon_number
glcp glc P
glcp glc P
glcp glc P
rhap rha P
rhap rha P
glcp1ome glc P
rhap rha P
missing_monosaccharide missing_monosaccharide missing_carbon_number
glcpnac glcnac P
glcp glc P
glcpnac glcnac P
galp gal P
glcp glc P
glcpn1po4 glc

In [12]:
all_table['bound_AB'] = ab_list
all_table['fischer_projection_DL'] = ld_list
all_table['origin_mono'] = abbrev_list
all_table['reformulated_standard_mono'] = standard_abbrev_list
all_table['carbon_number_PF'] = pf_list

In [13]:
all_table

,csv,pdb,glycan name,residual num,type,bound_AB,fischer_projection_DL,origin_mono,reformulated_standard_mono,carbon_number_PF
0,b-D-GlcpNAc,NAG,a-D-Kdop-(2-8)-a-D-Kdop-(2-4)-a-D-Kdop-(2-6)-b...,1,linear,b,d,glcpnac,glcnac,P
1,a-D-Kdop,KDO,a-D-Kdop-(2-8)-a-D-Kdop-(2-4)-a-D-Kdop-(2-6)-b...,2,linear,a,d,kdop,kdo,P
2,a-D-Kdop,KDO,a-D-Kdop-(2-8)-a-D-Kdop-(2-4)-a-D-Kdop-(2-6)-b...,3,linear,a,d,kdop,kdo,P
3,a-D-Kdop,KDO,a-D-Kdop-(2-8)-a-D-Kdop-(2-4)-a-D-Kdop-(2-6)-b...,4,linear,a,d,kdop,kdo,P
4,a-L-Fucp,FUC,repeat-2)-b-D-GlcpA-(1-3)-a-D-GalpNAc-(1-3)-a-...,1,linear,a,l,fucp,fuc,P
5,a-D-GalpNAc,A2G,repeat-2)-b-D-GlcpA-(1-3)-a-D-GalpNAc-(1-3)-a-...,2,linear,a,d,galpnac,galnac,P
6,a-D-GalpNAc,A2G,repeat-2)-b-D-GlcpA-(1-3)-a-D-GalpNAc-(1-3)-a-...,3,linear,a,d,galpnac,galnac,P
7,b-D-GlcpA,GLC,repeat-2)-b-D-GlcpA-(1-3)-a-D-GalpNAc-(1-3)-a-...,4,linear,b,d,glcpa,glca,P
8,b-D-Glcp,GLC,a-D-Xylp-(1-6)-b-D-Glcp-(1-4)-D-Glc-ol,1,linear,b,d,glcp,glc,P
9,a-D-Xylp,XYS,a-D-Xylp-(1-6)-b-D-Glcp-(1-4)-D-Glc-ol,2,linear,a,d,xylp,xyl,P


In [14]:
all_table.to_csv('matching_table/combined_all_formulated_monosaccharide_v2.csv', index = False)